In [2]:
import findspark
findspark.init()

import pyspark
from pyspark import sql

import pandas as pd

In [1]:
spark = SparkSession.\
    builder.\
    appName("MostPopularMovieWithRatingAbove4DataFrames").\
    getOrCreate()

In [2]:
path_ratings = r"/home/blake/PycharmProjects/BigDataMovies/the-movies-dataset/ratings.csv"
path_movies_metadata = r"/home/blake/PycharmProjects/BigDataMovies/the-movies-dataset/movies_metadata.csv"

ratings = spark.read.csv(path_ratings)
movies_metadata = spark.read.csv(path_movies_metadata)

In [ ]:
def header_dropper()

In [3]:
ratings

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string]